# Catalunya's TRIPS analysis in the COVID-19 restrictions

### Code contributors:
- Jordi Grau <jordi.grau@eurecat.org>
- Julià Vicens <julian.vicens@eurecat.org>

### Content
This notebook analyses the differences in Catalunya's mobility across the previously defined COVID-19 phases using visual analysis. In this notebook the focus is the NUMBER OF TRIPS. 

### Contents
0. [COVID-19 number of cases](#covid)

1. [Data querying](#data-querying)   
    1.1. [Trips](#data-querying-trips)   
    1.2. [Internal/incoming/outgoing trips](#data-querying-iio)

2. [Time Series](#ts)   
    2.1. [Trips evolution](#ts-trips)   
    2.2. [IIO trips evolution](#ts-iio)      

3. [Maps](#maps)     
    3.1. [Trips](#maps-trips)         
    
4. [Heatmaps](#heatmaps)       
    4.1. [Trips](#heatmap-trips)   
    
5. [Scatterplots](#Scatter)    
    5.1. [Mobilty vs covariants](#covariants-scatter)
    
6. [Boxplot](#Boxplots)   
    6.1 [Mobility by weekday](#mobility-weekday-boxplot)

In [ ]:
# Imports
import requests
import io
import datetime
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, r2_score
from sklearn import linear_model

from src.database_utils import *
from src.mobility_plots import *
from src.mobility_context_and_queries import *

# Date range to load raw data
date1 = '2020-03-02'
date2 = '2020-03-22'

In [ ]:
# Get all phases
print("All possible phases:\n")
all_phases = []
for phase in phases_list:
    phase_name, phase_period = list(phase.items())[0]
    print(f'- ({phase_name}, {phase_period["start"]}, {phase_period["end"]})')
    all_phases.append(phase_name)
    
#############################
############ FILL ###########
#############################

# Fill target phases wanted for the study
study_phases = [
    'precovid', 'lockdown', 'mobilitat_essenc', 'fase_0', 'desescalada', 
    'no_restriccions', 'alerta_5_inici', 'alerta_5_tr1', 'alerta_5_tr1_2', 'nadal_cap_reis']

# Print target phases names and dates
phases_dates_targets = []
for phase_dict in phases_list:
    for phase_name, dates in phase_dict.items():
        if phase_name in study_phases:
            phases_dates_targets.append((phase_name, dates['start'], dates['end']))
print('\nStudy about this phases:\n')
for study in phases_dates_targets:
    print(f'- {study}')

In [ ]:
# Get all motivs
print("All possible motivs:\n")
all_motivs = []
for motiv in motiv_list:
    motiv_name, motiv_period = list(motiv.items())[0]
    print(f'- ({motiv_name}, {motiv_period["start"]}, {motiv_period["end"]})')
    all_motivs.append(motiv_name)

#############################
############ FILL ###########
#############################

# Fill target motivs wanted for the study
study_motivs = []

# Print target motivs names and dates
motivs_dates_targets = []
for motiv_dict in motiv_list:
    for motiv_name, dates in motiv_dict.items():
        if motiv_name in study_motivs:
            motivs_dates_targets.append((motiv_name, dates['start'], dates['end']))
print('\nStudy about this motivs:\n')
for study in motivs_dates_targets:
    print(f'- {study}')

# 0. COVID-19 number of cases <a class="anchor" id="covid"></a>

In [ ]:
# https://datos.gob.es/es/catalogo/e05070101-evolucion-de-enfermedad-por-el-coronavirus-covid-19
df = pd.read_csv('casos_diagnostico_ccaa.csv')

# Filter catalunya
df = df[df['ccaa_iso'] == 'CT']
df['date'] = pd.to_datetime(df['fecha'], format='%Y-%m-%d')
df = df.set_index('date')['num_casos']

In [ ]:
fig, ax = plt.subplots(figsize=(14,4))
ax.set_title('COVID-19 pandemic evolution in Catalunya')
ax.set_ylabel('Number of cases')

df.plot(ax=ax)
ax.set_xlim(['2020-02-14', '2021-01-05'])
ax.grid()

# Restriction periods vertical lines and numbers
for i, phase in enumerate(phases_list):
    for name, dates in phase.items():
        # Middle date in restriction
        start = pd.to_datetime(dates['start'], format='%Y-%m-%d')
        end = pd.to_datetime(dates['end'], format='%Y-%m-%d')
        difference = start - end
        days = (int(str(difference).split(" ")[0].replace('-', '')))
        days_timedelta = timedelta(days=(days/2)-2)
        plot_date = pd.to_datetime(dates['start'], format='%Y-%m-%d') + days_timedelta
        
        if i == 0:
            ax.axvline(start, color='r', linestyle='--')
        ax.axvline(end, color='r', linestyle='--')
        #ax.plot(plot_date, 3000, '-o', ms=15, alpha=0.5, color='grey')
        ax.annotate(xy=[plot_date, 3000], text=i)

plt.show()

# 1. Data querying <a class="anchor" id="data-querying"></a>

## 1.1 Trips <a class="anchor" id="data-querying-trips"></a>

In [ ]:
# TRIPS RAW DATA query between 2 dates in Catalunya
table = 'mitma_cat_raw'
df_raw_data = query_raw_data_or_trips_or_flux_matrix_between_dates(
     table=table, date1=date1, date2=date2, province_groups=province_group)

# Describe dataframe
print(f"df_raw_data size: {df_raw_data.size}")
df_raw_data.head(3)

In [ ]:
# TRIPS MATRIX query between 2 dates in Catalunya
table = 'mitma_trips_matrix'
df_trips_matrix = query_raw_data_or_trips_or_flux_matrix_between_dates(
     table=table, date1=date1, date2=date2, province_groups=province_group)

# Describe dataframe
print(f"df_trips_matrix size: {df_trips_matrix.size}")
df_trips_matrix.head(3)

In [ ]:
# Daily trips for all days 
query = f"SELECT datetime, source, target, SUM(trips) FROM mitma_cat_raw \
            WHERE source = ANY(%(parameter_array)s) \
            GROUP BY datetime, source, target"

df_trips_all = query_parameters_cat(query, mitma_layers_cat)
df_trips_all['date'] = pd.to_datetime(df_trips_all['datetime'], format='%Y-%m-%d')

for location_target in location_info.keys():
    plot_ts_all_time(df_trips_all, location_target)

## 1.2 Internal/incoming/outgoing trips <a class="anchor" id="data-querying-iio"></a>

In [ ]:
# TRIPS IIO (incoming, internal, outgoing)
# query between 2 dates in Catalunya
table = 'mitma_trips'
df_trips_iio = query_trips_iio_or_qrp_between_dates(
    table=table, date1=date1, date2=date2, province_groups=province_group)

# Describe dataframe
print(f"df_trips_iio size: {df_trips_iio.size}")
df_trips_iio.head(3)

In [ ]:
# TRIPS HOURLY FLUX computation using raw data query
_, _, _, df_flux_hourly = compute_fluxes_by_area(
    df_raw_data, by_hour=True)

# Describe df
print(f"df_flux_hourly size: {df_flux_hourly.size}")
df_flux_hourly.head(3)

# 2. Time Series <a class="anchor" id="ts"></a>

## 2.1. Trips evolution <a class="anchor" id="ts-trips"></a>

In [ ]:
# DAILY and HOURLY raw trips between two dates
for target in location_info.keys():
    plot_ts_raw_trips(df_trips_all, target)

## 2.2. IIO trips evolution <a class="anchor" id="ts-iio"></a>

In [ ]:
# Hourly IIO trips. Location as source of trips
for location_target in location_info.keys():
    plot_ts_fluxes_mobility(df_flux_hourly, location_target, byhour=True, byweek=False)

In [ ]:
# Daily IIO trips. Location as source of trips
for location_target in location_info.keys():
    plot_ts_fluxes_mobility(df_trips_iio, location_target)

In [ ]:
# Weekly IIO trips. Location as source of trips
for location_target in location_info.keys():
    plot_ts_fluxes_mobility(df_trips_iio, location_target, byweek=True)

# 3. Maps <a class="anchor" id="maps"></a>

## 3.1. Trips <a class="anchor" id="maps-trips"></a>

In [ ]:
# Number of trips in a day
# Prepare studies
date_targets = ['20200303']
ori_desti = ['source'] # 'target'
metric_targets = ['trips'] #'viajes_km'

# Plot maps
for location_target in location_info.keys():
    for date_target in date_targets:
        for metric_target in metric_targets:
            for ori_target in ori_desti:
                plot_map_daily_trips(
                    df=df_raw_data, target=location_target, date=date_target, 
                    ori_desti=ori_target, metric=metric_target)

# 4. Heatmaps <a class="anchor" id="heatmaps"></a>

## 4.1. Trips  <a class="anchor" id="heatmap-trips"></a>

In [ ]:
# Trips by weekday and date
for location_target in location_info.keys():
    plot_htmap_raw_data(df_raw_data, target=location_target, metric='trips')

# 5. Scatterplot <a class="anchor" id="Scatter"></a>

In [ ]:
def plot_scatter_trips(df, phase1, phase2):

    # Filter mean mobility indexes by date
    df_date1 = df[df['phase'] == phase1]
    df_date2 = df[df['phase'] == phase2]
    df_date1.set_index('datetime', inplace=True)
    df_date2.set_index('datetime', inplace=True)

    # Sum one day's trips from one node to another
    data1 = df_date1.groupby(['source', 'target']).mean()['sum'].reset_index()
    data2 = df_date2.groupby(['source', 'target']).mean()['sum'].reset_index()

    # Merge both days' data to plot it
    df_merged = pd.merge(data1, data2, on=['source', 'target'], how='outer', suffixes=('_x', '_y')).fillna(0)
    x, y = np.array(df_merged['sum_x'].apply(round)).reshape(-1, 1), np.array(df_merged['sum_y'].apply(round)).reshape(-1, 1)

    difference = x.mean() - y.mean()
    print(f"Trips -> Mean difference: {round(difference, 2)} || " + \
        f"SME: {round(mean_squared_error(x, y), 2)} || " + \
        f"RSME: {round(mean_squared_error(x, y, squared=False), 2)} ||" + \
        f"R2: {round(r2_score(x, y), 2)}")

    # Scatter plot
    fig, ax =  plt.subplots(1,1, figsize=(6, 6))
    ax.scatter(x, y)
    ax.set_xlabel(phase1)
    ax.set_ylabel(phase2)
    fig.suptitle('Mean daily trips in all MITMA regions in Catalunya', fontsize=14)
    
    # 2 variables linear regression and R^2
    regr = linear_model.LinearRegression()
    regr.fit(x, y)
    plt.plot(x, regr.predict(x), color='red', linewidth=3)
    r2 = round(regr.score(y, x), 2)
    m = round(regr.coef_[0][0], 2)
    ax.set_title(f'(m: {m}, R2: {r2})')
    
    # Set axis limits
    max_value = max(max(x), max(y))
    max_value + max_value*0.05
    ax.set_ylim([0, max_value])
    ax.set_xlim([0, max_value])

    plt.tight_layout()
    plt.show()

In [ ]:
# df = df_trips_all.set_index('date').copy()

# # Add phases according to dates
# for phase in phases_list:
#     for phase_name, phase_dates in phase.items():
#         start, end = phase_dates['start'], phase_dates['end'] 
#         df.loc[(df.index >= start) & (df.index <= end), 'phase'] = phase_name

In [ ]:
# Add phases to dataframe
# add_phases_and_motivs(df_trips_all, phases_list, motiv_list)
for phase in phases_list[1:]:
    phase_name = list(phase.keys())[0]
    plot_scatter_trips(df, phase1='precovid', phase2=phase_name)

# 6. Boxplot <a class="anchor" id="Boxplot"></a>

## 6.1. Mobility by weekday <a class="anchor" id="mobility-weekday-boxplot"></a>

In [ ]:
def plot_boxplot_compare_two_periods_by_weekday(df_raw, phase1, phase2):

    df_raw = df_raw.reset_index()
    
    # First period. Group by weekday and source MITMA region 
    df = df_raw[df_raw['phase'] == phase1] 
    df['weekday'] = df['date'].dt.weekday
    print(sum(df['sum']))
    df = df.groupby(['weekday', 'source']).mean()
    df1 = df.reset_index()[['weekday', 'source', 'sum']]
    
    # Second period. Group by weekday and source MITMA region 
    df = df_raw[df_raw['phase'] == phase2] 
    df['weekday'] = df['date'].dt.weekday
    print(sum(df['sum']))
    df = df.groupby(['weekday', 'source']).mean()
    df2 = df.reset_index()[['weekday', 'source', 'sum']]
    
    # Boxplot legend
    df1['phase'] = phase1
    df2['phase'] = phase2

    # Reorder dataframe
    df = df1.append(df2)[['phase', 'weekday', 'sum']]
    df.columns = ['phase', 'weekday', 'mean daily trips']
    df = df.replace({'weekday' : {
        0:'Monday', 1:'Tuesday', 2:'Wednesday', 
        3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}})

    # Boxplot
    fig, ax = plt.subplots(figsize=(8,6))
    sns.violinplot(data=df, x='weekday', y="mean daily trips", hue="phase", ax=ax)
    ax.set_title('Mean daily trips in all MITMA regions in Catalunya', fontsize=16)
    plt.show()

In [ ]:
# Remove warning
pd.options.mode.chained_assignment = None  # default='warn'

# Boxplot for each phase
for phase in phases_list[1:]:
    phase_name = list(phase.keys())[0]  
    plot_boxplot_compare_two_periods_by_weekday(df, phase1='precovid', phase2=phase_name)